In [ ]:
1

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from openfisca_france import CountryTaxBenefitSystem
from openfisca_core.reforms import Reform

In [ ]:
import sys

sys.path.append("../technique")

In [ ]:
from dotenv import load_dotenv

load_dotenv()
import os

In [ ]:
from utils import *

In [ ]:
from scenario import CEReform, new_bracket

In [ ]:
from mobilite import add_compensation, build_data, compute_result, get_df, get_results

In [ ]:
base = CountryTaxBenefitSystem()
base.load_extension("openfisca_france_local")

In [ ]:
qf_fiscal_reform = QfFiscalReform(base)

In [ ]:
statut_reform = StatutReform(qf_fiscal_reform)

In [ ]:
pd.options.display.float_format = lambda v: "{:_.2f}".format(v).replace("_", " ")

In [ ]:
recap_f, [(_, res_f)] = get_results(base, sample_count=10, reform=qf_fiscal_reform)

In [ ]:
res_f.groupby(["sample_id", "titre_fichier"]).ajustement_mensuel_num.sum().groupby(
    "titre_fichier"
).describe()

In [ ]:
res_f.groupby(["sample_id", "titre_fichier"]).pu_calc_ht.sum().groupby(
    "titre_fichier"
).median()

In [ ]:
recap_f

In [ ]:
quantite_ancien_qf = (
    res_f.groupby(["sample_id", "pu_calc"])
    .ajustement_mensuel_num.sum()
    .groupby("pu_calc")
    .median()
)
quantite_ancien_qf

In [ ]:
quantite_nouveau_qf = (
    res_f.groupby(["sample_id", "pu_calc_r"])
    .ajustement_mensuel_num.sum()
    .groupby("pu_calc_r")
    .median()
)
quantite_nouveau_qf

In [ ]:
res_f[~res_f.ok]

In [ ]:
assert [*res_f[~res_f.ok].titre_fichier.unique()] == [" +65 Annuel", "PMR Gratuit"]

In [ ]:
res_f[res_f.titre_fichier.isin([*res_f[~res_f.ok].titre_fichier.unique()])][
    ["sample_id", "titre_fichier", "pu_fichier", "pu_calc_ht"]
].groupby(["sample_id", "titre_fichier"]).sum().groupby("titre_fichier").describe()

In [ ]:
ecarts = (
    res_f[res_f.titre_fichier.isin([*res_f[~res_f.ok].titre_fichier.unique()])][
        [
            "sample_id",
            "titre_fichier",
            "ajustement_mensuel_num",
            "pu_fichier",
            "pu_calc_ht",
        ]
    ]
    .rename(
        columns={
            "ajustement_mensuel_num": "quantité ajustée",
            "pu_fichier": "recettes fichier",
            "pu_calc_ht": "recettes recalculées",
        }
    )
    .groupby(["sample_id", "titre_fichier"])
    .sum()
    .groupby("titre_fichier")
    .agg(["mean", "std"])
)
ecarts

In [ ]:
ecarts.to_excel("ecarts.xlsx", sheet_name="Écarts")

In [ ]:
field = "pu_calc"
# field = "pu_fichier"
pd.pivot_table(
    res_f[[field, field + "_r", "quantité"]],
    columns=field + "_r",
    index=field,
    values="quantité",
    aggfunc=len,
)

In [ ]:
res_f

In [ ]:
res_f[[field, field + "_r", "quantité"]]

In [ ]:
res_f[field + "_rs"] = res_f[field + "_r"].astype("str")
suffix = "_r"
count_field = "quantité"
# count_field = "ajustement_mensuel_num"
matrice = (
    pd.pivot_table(
        res_f[[field, field + suffix, count_field, "sample_id"]],
        columns=field + suffix,
        index=[field, "sample_id"],
        values=count_field,
        aggfunc=sum,
        fill_value=0,
    )
    .reset_index(field)
    .groupby([field])
    .median()
)

In [ ]:
matrice

In [ ]:
matrice.columns = pd.Index(
    matrice.columns.values.astype("float64").round(2)
)  # .astype('str'))

In [ ]:
matrice / 12

In [ ]:
matrice.sum().sum()

In [ ]:
# matrice.to_excel('matrice_prix.xlsx')

In [ ]:
res_f.groupby("sample_id").quantité.sum()

In [ ]:
res_f["delta_p"] = res_f.pu_calc_r - res_f.pu_calc

In [ ]:
res_f.groupby(["pu_calc", "pu_calc_r"]).quantité.count().apply(
    np.sqrt
).reset_index().plot.scatter(x="pu_calc", y="pu_calc_r", s="quantité")

In [ ]:
df, compens_constant = get_df()

In [ ]:
rdf = pd.DataFrame(
    data={
        "pu_fichier": df.PU,
        "pu_calc": df.PU * 0,
        "tp": df.PU * 0,
    }
)
data, complement_df = build_data(df, rdf, 30)

In [ ]:
complement_df["qf_caf"] = data["input_data_frame_by_entity"]["famille"]["qf_caf"]
complement_df["qf_fiscal"] = data["input_data_frame_by_entity"]["famille"]["qf_fiscal"]

In [ ]:
prix_max = res_f.pu_calc.max()
count_max = (
    res_f[(res_f.pu_calc == prix_max)]
    .groupby("sample_id")
    .ajustement_mensuel_num.sum()
    .median()
)

In [ ]:
index_max = (df.quantité.sum() - count_max + 1).astype("int64")
ranks = (
    complement_df.groupby("sample_id").qf_fiscal.rank(method="first").astype("int64")
)
ranks_max = (
    complement_df.groupby("sample_id").qf_fiscal.rank(method="max").astype("int64")
)
complement_df.qf_fiscal[ranks[ranks == index_max].index].describe()

In [ ]:
complement_df[ranks == 1]

In [ ]:
count_total = len(res_f[res_f.sample_id == 0])
count_total

In [ ]:
counts_zero = ranks_max[ranks == 1]
counts_restants = count_total - counts_zero - count_max

In [ ]:
cr = counts_restants.median()
cr

In [ ]:
counts_zero.median()

In [ ]:
res_f["qf_fiscal_nul"] = res_f.qf_fiscal == 0
count_min = (
    res_f[res_f.qf_fiscal == 0]
    .groupby("sample_id")
    .ajustement_mensuel_num.sum()
    .median()
)
prix_min = res_f.pu_calc[res_f.pu_calc > 0].min()

In [ ]:
recettes_totales = (res_f.ajustement_mensuel_num * res_f.pu_calc).sum()
recettes_totales

In [ ]:
recettes_restantes = recettes_totales - count_max * prix_max - count_min * prix_min
recettes_restantes

In [ ]:
for i in range(1, 6, 2):
    steps = [*range(1, i + 1)]
    cell_m = round(recettes_restantes / sum(steps))
    # linéaire
    print((i, [cell_m * c for c in steps]))
    #

In [ ]:
count_restant = res_f.ajustement_mensuel_num.sum() - count_max - count_min
count_restant

In [ ]:
recettes_restantes / count_restant

In [ ]:
complement_df["qf_fiscal_nul"] = complement_df.qf_fiscal == 0
complement_df.groupby("sample_id").qf_fiscal_nul.sum().describe()

In [ ]:
for i in range(1, 6, 2):
    steps = [*range(1, i + 1)]
    cell_m = round(recettes_restantes / sum(steps))
    print((i, [cell_m * c for c in steps]))

In [ ]:
recap_s, [(_, res_s)] = get_results(base, sample_count=1, reform=statut_reform)

In [ ]:
recap_s

In [ ]:
res_s["pu_calc_rs"] = res_s.pu_calc_r.round(2).astype("str").astype("category")

In [ ]:
rebase = CountryTaxBenefitSystem()
rebase.load_extension("openfisca_france_local")
re_qf_fiscal_reform = QfFiscalReform(rebase)

In [ ]:
def f_replace(local_parameters):
    bfs = [
        local_parameters.metropoles.strasbourg.tarification_solidaire.bareme,
        local_parameters.metropoles.strasbourg.tarification_solidaire.bareme_reduit,
    ]
    for b in bfs:
        brackets = [new_bracket(0, 3.4), new_bracket(1, 20), new_bracket(1000, 51.8)]
        bbrackets = [
            new_bracket(0, 3.4),
            new_bracket(1, 5.8),
            new_bracket(300, 14.4),
            new_bracket(500, 25.6),
            new_bracket(1000, 51.8),
        ]
        b.brackets = brackets

    return local_parameters


class FCustomReform(Reform):
    def __init__(self, tbs):
        super().__init__(tbs)

    def apply(self):
        self.modify_parameters(modifier_function=f_replace)


re_max_reform = FCustomReform(re_qf_fiscal_reform)
re_statuts_reform = StatutReform(re_max_reform)
# re_statuts_reform = StatutReform(re_grist_reform)
re_ce_reform = CEReform(re_statuts_reform)
full_reform = re_ce_reform
full_reform = re_ce_reform

In [ ]:
recap_r, [(_, res_r)] = get_results(base, sample_count=10, reform=full_reform)

In [ ]:
res_r

In [ ]:
res_r[res_r.qf_fiscal == 0].groupby("sample_id").ajustement_mensuel_num.sum().median()

In [ ]:
if False:
    writer = pd.ExcelWriter("tables_mobilité.xlsx")
    df_qf = pd.DataFrame(
        {"quantité QF CAF": quantite_ancien_qf, "quantité QF EMS": quantite_nouveau_qf},
        index=quantite_ancien_qf.index,
    )
    df_qf.to_excel(writer, sheet_name="Prix")
    qf_nul = pd.DataFrame(
        {
            "quantité": [
                res_r[res_r.qf_fiscal == 0]
                .groupby("sample_id")
                .ajustement_mensuel_num.sum()
                .median()
            ]
        }
    )
    qf_nul.to_excel(writer, sheet_name="QF_EMS_nul")
    writer.close()

In [ ]:
field = "pu_calc"
# field = "pu_fichier"
pd.pivot_table(
    res_f[[field, field + "_r", "quantité"]],
    columns=field + "_r",
    index=field,
    values="quantité",
    aggfunc=len,
)

In [ ]:
res_f

In [ ]:
prix_calc_df = (
    res_f.groupby(["pu_calc", "sample_id"])
    .quantité.count()
    .reset_index(name="quantité")
    .rename(columns={"pu_calc": "prix"})
)
prix_calc_r_df = (
    res_f.groupby(["pu_calc_r", "sample_id"])
    .quantité.count()
    .reset_index(name="quantité")
    .rename(columns={"pu_calc_r": "prix"})
)

prix_calc_df["scenario"] = "c"  #'qf_caf'
prix_calc_r_df["scenario"] = "f"  #'qf_fiscal'

In [ ]:
prix_df = pd.concat([prix_calc_df, prix_calc_r_df])
prix_df.prix = prix_df.prix.round(2)

In [ ]:
prix_df.groupby(["prix", "scenario"]).quantité.median()

In [ ]:
prix_calc_df

In [ ]:
prix_calc_r_df

In [ ]:
recap_statut, [(_, res_statut)] = get_results(
    base, sample_count=10, reform=statut_reform
)

In [ ]:
recap_statut

In [ ]:
if False:
    count = df.quantité.sum().astype("int64")
    static_sample_count = 3
    static_sample_ids = np.repeat(list(range(static_sample_count)), count)
    static_sample_individu_df = pd.DataFrame(
        {
            "sample_id": static_sample_ids,
            "famille_id": list(range(count * static_sample_count)),
            "qfrule": np.tile(np.repeat(df.QF, df.quantité), static_sample_count),
            "agerule": np.tile(np.repeat(df.AGE, df.quantité), static_sample_count),
            "taux_incapacite": np.tile(
                np.repeat(np.where(df.Titres.str.contains("PMR"), 0.8, 0), df.quantité),
                static_sample_count,
            ),
        }
    )

    determine_qf(static_sample_individu_df)

    rr = static_sample_individu_df.groupby(by="sample_id").qf_fiscal.rank(
        method="first"
    )

    df[["QF", "quantité"]].groupby(by="QF").sum()

    indexes = rr.isin(df[["QF", "quantité"]].groupby(by="QF").sum().cumsum().quantité)
    rr_res = pd.DataFrame(
        data={
            "rank": rr[indexes],
            "value": static_sample_individu_df.qf_fiscal[indexes],
        }
    )

    rr_res.groupby(by="rank").describe()

    no_reduit_indexes = rr.isin(merge_bareme_population)
    no_reduit_rr_res = pd.DataFrame(
        data={
            "rank": rr[no_reduit_indexes],
            "value": static_sample_individu_df.qf_fiscal[no_reduit_indexes],
        }
    )

    no_reduit_rr_res_desc = no_reduit_rr_res.groupby(by="rank").value.describe()
    no_reduit_rr_res_desc

    no_reduit_rr_res_desc["50%"].round().astype("int64")